In [ ]:
source(paste0(dirname(dirname(dirname(getwd()))),'/map.r'))
source(paste0(HELP_DIR, "shortcuts.r"))
source(paste0(HELP_DIR, "helpers.r"))

In [ ]:
library(patchwork)

# 0 - Ready prepared cohort counts

In [ ]:
top_mechanisms <- fread(paste0(SHARE_DIR, "top_mechanisms.csv"))

In [ ]:
top_pan <- top_mechanisms %>% fi(grepl("Pan-Cancer", cohortGo)) 
top_tissue <- top_mechanisms %>% fi(grepl("Pan-Cancer", cohortGo)) 

In [ ]:
p_base <- 
top_mechanisms %>% 
 mu(cohortGoo = ifelse(cohortGo == "Pan-Cancer", "All treatments", cohortGo)) %>% 
 mu(pan = grepl("Pan-Cancer", cohortGo) | cohortGo == "All treatments", 
    cohortGoo = gsub("Pan-Cancer ## ", "", cohortGoo), 
    cohortGoo = gsub("\\(ant\\)agonist", "", cohortGoo), 
    cohortGoo = gsub(" ## ", " / ", cohortGoo)) %>% 
 mu(cohortGooo = ifelse(!pan, sub(" / ", "\n", cohortGoo), cohortGoo))

In [ ]:
#p_base

In [ ]:
fwrite( p_base , paste0(SHARE_DIR, "cohort_names_map.csv"))

In [ ]:
base <- 
p_base %>% 
 ga( response, val, -cohortGo, -group, -ct, -pan, -cohortGoo, -cohortGooo) %>% 
 mu(response = factor(ifelse(response == "no_dcb", "No DCB", "DCB"), levels = rev(c("DCB", "No DCB"))),
    cohortGo = factor(cohortGo, levels = p_base$cohortGo[order(p_base$ct)]), 
    cohortGoo = factor(cohortGoo, levels = p_base$cohortGoo[order(p_base$ct)]),
    group = factor(group, levels = c("treatment", "mechanism", "type"))) %>% 
 group_by(cohortGo, cohortGoo) %>%
 arrange(cohortGo, rev(response)) %>%
 mutate(pos = cumsum(val) - 0.5 * val)

# 1 - Figure

- Figure function

In [ ]:
colors <- c("No DCB" = "#e52f28", "DCB" = "#7AABD3")

In [ ]:
extra_theme <- 
theme(legend.position = "none",
    axis.title.x = element_blank(),
    axis.title.y = element_blank(),
    axis.text.y = element_text(size = 12), 
    axis.text.x = element_text(size = 12, angle = 0), 
    plot.title = element_text(size = 16) ) 

In [ ]:
plter <- function(d_base, title = "X", text_size = 4){
 df_totals <- d_base %>% gb(cohortGoo) %>% su(total = sum(val)) 
    
 #d_base <- d_base %>% mu(response = ifelse(response == "no_dcb", "No DCB", "DCB"))

 ggplot(d_base, aes(x = val, y = cohortGoo, fill = response)) +
  geom_bar(stat = "identity", color = "black", alpha = .8) +
  labs(title = title) +
  geom_text(aes(label = val, x= pos), color = "black", position = position_dodge(), size = text_size) + 
  scale_fill_manual( values = colors) +  
  geom_text(data = df_totals, aes(y = cohortGoo, x = total, label = total), inherit.aes = FALSE, hjust = -.2, size = 4) + 
  scale_x_continuous(expand = expansion(c(0, 0.12))) + 
  go_theme + extra_theme
}

- Go go plots

In [ ]:
p1 <- plter(base %>% fi(!pan, group == "treatment"), "Cohort Specific: Drug Treatment")
p2 <- plter(base %>% fi(!pan, group == "mechanism"), "Cohort Specific: Drug Mechanism") 
p3 <- plter(base %>% fi(!pan, group == "type"), "Cohort Specific: Drug Type") + labs(title = "Cohort Specific: Drug Type", x = "Number of Patients") + go_theme + extra_theme + theme(legend.position = "none",axis.title.y = element_blank()) 
p4 <- plter(base %>% fi(pan, group == "treatment"), "Pan-Cancer: Drug Treatment") + theme(legend.position = c(0.7, 0.25) ) + labs(fill = NULL)
p5 <- plter(base %>% fi(pan, group == "mechanism"), "Pan-Cancer: Drug Mechanism") 
p6 <- plter(base %>% fi(pan, group == "type"), "Pan-Cancer: Drug Type")  + labs(title = "Pan-Cancer: Drug Type", x = "Number of Patients") + go_theme + extra_theme + theme(legend.position = "none",axis.title.y = element_blank()) + scale_x_continuous(expand = expansion(c(0, 0.2))) 

- Drug mechanism

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 8)

In [ ]:
cohort <- (p1 / p2 / p3 ) + plot_layout(heights = c(5, 7, 8)) 
pan_cancer <- (p4 / p5 / p6 ) + plot_layout(heights = c(6, 6, 3)) 
cohorts_full = (cohort | pan_cancer ) + plot_layout(widths = c(1,1)) 

In [ ]:
treatment <- (p1 / p4 ) + plot_layout(heights = c(3,5)) 
mechanism <- (p2 / p5) + plot_layout(heights = c(3,3)) 

In [ ]:
cohorts_reduced = (treatment | mechanism ) + plot_layout(widths = c(1,1)) 

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 11)
#cohorts_reduced
cohorts_full

In [ ]:
ggsave( paste0(SHARE_DIR, "0_cohorts_reduced.png"), plot = cohorts_reduced, width = 14, height = 8)
ggsave( paste0(SHARE_DIR, "0_cohorts_full.png"), plot = cohorts_full, width = 14, height = 11)